# Get songs from one of the lyrics websites and save them in a text file
I now have a dictionary with albums and song titles saved as a pickle. I need to go to one of the lyrics pages, find song lyrics for each album and save them in a text file. Fist I need to load my pickle file and check my song titles are there.

In [1]:
import pickle
import os
pathCore = "C:\\Users\\karol\\jupyter_startUp_dir\\TheNationalSongMaker"
os.chdir(pathCore)
with open('theNationalSongDictionary.pickle', 'rb') as handle:
    theNationalSongDictionary = pickle.load(handle)
    

In [3]:
print(theNationalSongDictionary)

{'The Virginia EP': ['"You\'ve Done It Again, Virginia"', '"Santa Clara"', '"Blank Slate"', '"Tall Saint"', '"Without Permission"', '"Forever After Days"', '"Rest of Years"', '"Slow Show"', '"Lucky You"', '"Mansion on the Hill"', '"Fake Empire"', '"About Today"'], 'Cherry Tree': ['"Wasp Nest"', '"All the Wine"', '"All Dolled-Up in Straps"', '"Cherry Tree"', '"About Today"', '"Murder Me Rachael"', '"A Reasonable Man (I Don\'t Mind)"'], 'Sad Songs for Dirty Lovers': ['"Cardinal Song"', '"Slipping Husband"', '"90-Mile Water Wall"', '"It Never Happened"', '"Murder Me Rachael"', '"Thirsty"', '"Available"', '"Sugar Wife"', '"Trophy Wife"', '"Fashion Coat"', '"Patterns of Fairytales"', '"Lucky You"'], 'Alligator': ['"Secret Meeting"', '"Karen"', '"Lit Up"', '"Looking for Astronauts"', '"Daughters of the SoHo Riots"', '"Baby, We\'ll Be Fine"', '"Friend of Mine"', '"Val Jester"', '"All the Wine"', '"Abel"', '"The Geese of Beverly Road"', '"City Middle"', '"Mr. November"'], 'Boxer': ['"Fake Empi

OK,we have a dictionary with all the songs now. Let's get all the lyrincs from www.lyrics.com.
Use beutiful soup, ?selenium?
First go to The National page:
https://www.lyrics.com/artist/The-National/13762

Find songs matching dictionary values, click and go to page, scrape

In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup

url = "https://www.lyrics.com/artist/The-National/13762" 

#####

driver = webdriver.Chrome("C:/Users/karol/drivers/chromedriver")  #select selenium web driver
driver.implicitly_wait(10)
driver.get(url) #open the url in selenium

#link = driver.find_element_by_link_text('Details')

link = driver.find_element_by_link_text("29 Years").click()


#print(link)

soup = BeautifulSoup(driver.page_source,'html5lib') #grab the content with beautifulsoup for parsing
    #main_table = soup.findAll("pre",{"class":"item"}) 
div = soup.find(id="lyric-body-text").text
print(div)
    
    #id =lyric-body-text

Could you tell re-enchantment I said goodbye
I met a girl named disillusionment
Could you tell her I made the headlines
This is a boy who draws wings on everything

Could you tell her to check the headlines
This is a boy who finds angels in the river
You know I dreamed about you
For twenty-nine years before
I saw you

You know I dreamed about you
I missed you for twenty-nine years
Birds flyin' out of order
Underneath the sky

I'll run up to the rainbow girl
Just to pass her by
And I'll never have a change of heart
I've sworn I'd never sing

I have no heart this one is gone
And now I wear the wings
You know I dreamed about you
For twenty-nine years before
I saw you

You know I dreamed about you (twenty-nine years)
I missed you for twenty-nine years
You know I dreamed about you
For twenty-nine years before I saw you

You know I dreamed about you
For twenty-nine years 
Before I saw you


# # After some experimentation with the https://www.lyrics.com I discovered some of the songs are missing. I found a site with a more complete list at: https://www.azlyrics.com

Songs for each album will be saved in separate folders named after album name. We will need to create titles for folders with songs.

In [24]:
#for each album in a list ceate a string,
#capitalise all words and remove spaces
albumNum = 0
for albumNum in range(0, len(theNationalSongDictionary)):
    print(list(theNationalSongDictionary.keys())[albumNum].title().replace(" ", ""))


TheVirginiaEp
CherryTree
SadSongsForDirtyLovers
Alligator
Boxer
HighViolet
TheNational
TroubleWillFindMe
SleepWellBeast


In [4]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import string
import re
import os

def title_one_liner(title):
    return re.sub(r"(?<=[a-z])[\']([A-Z])", lambda x: x.group().lower(), title.title())
url = "https://www.azlyrics.com/n/national.html" 

#####

driver = webdriver.Chrome("C:/Users/karol/drivers/chromedriver")  #select selenium web driver
driver.implicitly_wait(20)
driver.get(url) #open the url in selenium

pathCore = "C:\\Users\\karol\\jupyter_startUp_dir\\TheNationalSongMaker"

albumNum = 0
for albumNum in range(0, len(theNationalSongDictionary)):
    albumTitle = list(theNationalSongDictionary.keys())[albumNum].title().replace(" ", "")
    albumSongs =list(theNationalSongDictionary.values())[albumNum]
    #print(list(theNationalSongDictionary.keys())[albumNum].title().replace(" ", ""))
#pathToAlbumFolder = pathCore + "\\" + "theVirginiaEP"
    pathToAlbumFolder = pathCore + "\\" + albumTitle
    print(pathToAlbumFolder)
    
    if os.path.exists(pathToAlbumFolder): #if the folder exists
        print("exists")
    elif not os.path.exists(pathToAlbumFolder):
        os.makedirs(pathToAlbumFolder)
        os.chdir(pathToAlbumFolder)



#for song in album:
        for song in albumSongs:
            #print(song)
            song_no_quotes = (song.strip('\"'))  #need to remove quotes
            song_no_quotes=song_no_quotes.split('(')[0] #remove anything in brackets, eg in A Reasonable Man (I Don'T Mind)

            #PROBLEM we want to not capitalise after hyphenation, but do want to capitalise after a dash
            #solutin here: https://daviseford.com/blog/2017/04/27/python-string-to-title-including-punctuation.html

            #song_no_quotes =string.capwords(song_no_quotes)#need to capitalise everything, articles in my list are not capitalised, and in the page are
            #song_no_quotes =song_no_quotes.title()#unlike capwords, will capitalise after hyphenation, we don't want that

            song_no_quotes=title_one_liner(song_no_quotes)

            #special rule for Reasonable Man, (a different string on website: Reasonable man instead a reasonable man)
            #maybe I should remove all As, let's try
            #
            #song_no_quotes = (song_no_quotes.replace("A ","")) was messing the Startr A War song title
            #another exception is Slow Show[25], the naming is not consistent, so I need to hard code some titles
            if song_no_quotes == "Slow Show[25]":
                song_no_quotes = "Slow Show"
            elif song_no_quotes == "A Reasonable Man":
                song_no_quotes = "Reasonable Man"
            elif song_no_quotes == "I'Ll Still Destroy You":
                song_no_quotes = "I'll Still Destroy You"


            print(song_no_quotes)

            link = driver.find_element_by_partial_link_text(song_no_quotes).get_attribute('href')  

            print(link)


            page_response = requests.get(link, timeout=10)
        # here, we fetch the content from the url, using the requests library
            soup = BeautifulSoup(page_response.content, "html.parser")

            lenght = len(soup.findAll("div", attrs={'class': None}))
            print(lenght)
            for i in range(0, lenght):
                paragraphs = soup.findAll("div", attrs={'class': None})[i].text
                if len(paragraphs)>0: #chec there is anything in the string

                    filename = ''.join(song_no_quotes.split()).lower()
                    f= open(filename,"w+")#first we get an empty string

                    f.write(paragraphs.strip())
                    f.close() 
#         <a href="../lyrics/national/youvedoneitagainvirginia.html" target="_blank">You've Done It Again, Virginia</a>
#         You've Done It Again, Virginia
#         You've Done It Again, Virginia
#         "You've Done It Again, Virginia"
        break #remove break if you want to run everything at once
        
        


C:\Users\karol\jupyter_startUp_dir\TheNationalSongMaker\TheVirginiaEp
exists
C:\Users\karol\jupyter_startUp_dir\TheNationalSongMaker\CherryTree
exists
C:\Users\karol\jupyter_startUp_dir\TheNationalSongMaker\SadSongsForDirtyLovers
exists
C:\Users\karol\jupyter_startUp_dir\TheNationalSongMaker\Alligator
exists
C:\Users\karol\jupyter_startUp_dir\TheNationalSongMaker\Boxer
Fake Empire
https://www.azlyrics.com/lyrics/national/fakeempire.html
2
Mistaken For Strangers
https://www.azlyrics.com/lyrics/national/mistakenforstrangers.html
2
Brainy
https://www.azlyrics.com/lyrics/national/brainy.html
2
Squalor Victoria
https://www.azlyrics.com/lyrics/national/squalorvictoria.html
2
Green Gloves
https://www.azlyrics.com/lyrics/national/greengloves.html
2
Slow Show
https://www.azlyrics.com/lyrics/national/slowshow.html
2
Apartment Story
https://www.azlyrics.com/lyrics/national/apartmentstory.html
2
Start A War
https://www.azlyrics.com/lyrics/national/startawar.html
2
Guest Room
https://www.azlyrics.c

The above code will create folders with lyrics, saved as txt files. When running the script I got blocked by the https://www.azlyrics.com website. I tried using a private ip address but was still getting blocked, so I used ProtonVPN (free version), to be able to complete my song download.

In [ ]:
The above code does everything I need, below are my attempts before reaching the final version.

In [32]:
#try scraping one song and print it
from bs4 import BeautifulSoup
import requests

page_link = "https://www.azlyrics.com/lyrics/national/bloodbuzzohio.html"
#page_link = "https://www.azlyrics.com/lyrics/national/sorrow.html"
# this is the url that we've already determined is safe and legal to scrape from.
page_response = requests.get(page_link, timeout=5)
# here, we fetch the content from the url, using the requests library
page_content = BeautifulSoup(page_response.content, "html.parser")
#we use the html parser to parse the url content and store it in a variable.
textContent = []
# for i in range(0, 20):
#     paragraphs = page_content.findAll("div", attrs={'class': None})[i].text
#     print(paragraphs)
#count paragraphs
lenght = len(page_content.findAll("div", attrs={'class': None}))
print(lenght)
for i in range(0, lenght):
    paragraphs = page_content.findAll("div", attrs={'class': None})[i].text
   
    if len(paragraphs)>0: #chec there is anything in the string
        print(type(paragraphs))
        #print(paragraphs)
        print(paragraphs.strip())
        print(len(paragraphs.lstrip()))
        
   
        new = (paragraphs.splitlines())[2:] #show all extra lines
        #print(paragraphs.split('\n')[2:])
       # paragraphs.splitlines()[1]
        f= open("song2.txt","w+")#first we get an empty string
 
        f.write(paragraphs.strip())
     #   f.write(str(new))
        f.close() 
    #this works

2
<class 'str'>
Stand up straight at the foot of your love
I'll lift my shirt up
Stand up straight at the foot of your love
I'll lift my shirt up

I was carried to Ohio in a swarm of bees
I never married but Ohio don't remember me

Lay my head on the hood of your car
I take it too far
Lay my head on the hood of your car
I take it too far

I still owe money to the money to the money I owe
I never thought about love when I thought about home
I still owe money to the money to the money I owe
The floors are falling out from everybody I know

I'm on a bloodbuzz
Yes I am
I'm on a blood...buzz
I'm on a bloodbuzz
God I am
I'm on a blood...buzz

I was carried to Ohio in a swarm of bees
I never married but Ohio don't remember me

I still owe money to the money to the money I owe
I never thought about love when I thought about home
I still owe money to the money to the money I owe
The floors are falling out from everybody I know

I'm on a bloodbuzz
Yes I am
I'm on a blood...buzz
I'm on a bloodbuz

In [7]:
#getting the path right
import os
pathCore = "C:\\Users\\karol\\jupyter_startUp_dir\\TheNationalSongMaker"
pathToAlbumFolder = pathCore + "\\" + "theVirginiaEP"
print(pathToAlbumFolder)

if not os.path.exists(pathToAlbumFolder):
    os.makedirs(pathToAlbumFolder)
os.chdir(pathToAlbumFolder)
cwd = os.getcwd()
print(cwd)

C:\Users\karol\jupyter_startUp_dir\TheNationalSongMaker\theVirginiaEP
C:\Users\karol\jupyter_startUp_dir\TheNationalSongMaker\theVirginiaEP


In [2]:
text = "A Reasonable Man (I Don'T Mind)"
text1= text.split('(')[0]
print(text1)

A Reasonable Man 


In [5]:
import string
print(string.capwords("I don't mind the"))

I Don't Mind The


In [14]:
import re

def lowercase_match_group(matchobj):
    return matchobj.group().lower()

def title_extended(title):
    if title is not None:
        # Take advantage of title(), we'll fix the apostrophe issue afterwards
        title = title.title()

        # Special handling for contractions
        poss_regex = r"(?<=[a-z])[\']([A-Z])"
        title = re.sub(poss_regex, lowercase_match_group, title)

    return title

def title_one_liner(title):
    return re.sub(r"(?<=[a-z])[\']([A-Z])", lambda x: x.group().lower(), title.title())

str12 = "I don't mind the dolled-up"
print(title_extended(str1))

def title_one_liner(title):
    return re.sub(r"(?<=[a-z])[\']([A-Z])", lambda x: x.group().lower(), title.title())
print(title_one_liner(str12))

I Don't Mind The Dolled-Up
I Don't Mind The Dolled-Up


In [22]:
#trying to use different user agents and ip addresses to get around website blocking scraping, eventually used a VPN

#driver = webdriver.Chrome("C:/Users/karol/drivers/chromedriver", options.add_argument('--proxy-server={170.81.37.34}'.format(proxy)))

from fake_useragent import UserAgent
from selenium import webdriver

ua = UserAgent()
userAgent = ua.random
print(userAgent)

PROXY = "200.255.122.174:8080" # IP:PORT or HOST:PORT

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--proxy-server=%s' % PROXY)
chrome_options.add_argument(f'user-agent={userAgent}')

chrome = webdriver.Chrome("C:/Users/karol/drivers/chromedriver",options=chrome_options)
#chrome.get("https://www.azlyrics.com/lyrics/national/bloodbuzzohio.html")
chrome.get("https://www.azlyrics.com/")

                            
  